## Loan default prediction

In [5]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing,metrics 
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import linear_model,svm
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [33]:
# load data
loans = pd.read_csv('/Users/Muyan/Desktop/loans_clean.csv', low_memory=False)

In [34]:
loans.head()

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,issue_d,loan_status,url,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,policy_code,application_type,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mort_acc,pub_rec_bankruptcies
0,68407277,3600.0,36 months,13.99,123.03,5,C4,leadman,6,55000.0,Not Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,debt consolidation,190xx,PA,5.91,Aug-2003,7.0,0.0,2765.0,29.7,13.0,w,1.0,Individual,148.0,128.0,1.0,0.0
1,68355089,24700.0,36 months,11.99,820.28,5,C1,engineer,6,65000.0,Not Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,small_business,business,577xx,SD,16.06,Dec-1999,22.0,0.0,21470.0,19.2,38.0,w,1.0,Individual,113.0,192.0,4.0,0.0
2,68341763,20000.0,60 months,10.78,432.66,6,B4,truck driver,6,63000.0,Not Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,unknown,605xx,IL,10.78,Aug-2000,6.0,0.0,7869.0,56.2,18.0,w,1.0,Joint App,125.0,184.0,5.0,0.0
3,68476807,10400.0,60 months,22.45,289.91,2,F1,contract specialist,6,104433.0,Source Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,major purchase,174xx,PA,25.37,Jun-1998,12.0,0.0,21929.0,64.5,35.0,w,1.0,Individual,128.0,210.0,6.0,0.0
4,68426831,11950.0,36 months,13.44,405.18,5,C3,veterinary tecnician,5,34000.0,Source Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,debt consolidation,300xx,GA,10.20,Oct-1987,5.0,0.0,8822.0,68.4,6.0,w,1.0,Individual,338.0,54.0,0.0,0.0


In [35]:
loans=loans.dropna(thresh = 340000,axis=1)

In [36]:
loans = loans[(loans['loan_status'] == 'Fully Paid') | (loans['loan_status'] == 'Charged Off') | (loans['loan_status'] == 'Default')]
loans = loans.assign(paid_flag = np.where(loans['loan_status'].str.contains('Fully Paid'), 1, 0))

In [37]:
loans.head()

,id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,home_ownership,annual_inc,verification_status,issue_d,loan_status,url,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,policy_code,application_type,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mort_acc,pub_rec_bankruptcies,paid_flag
0,68407277,3600.0,36 months,13.99,123.03,5,C4,leadman,6,55000.0,Not Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,debt consolidation,190xx,PA,5.91,Aug-2003,7.0,0.0,2765.0,29.7,13.0,w,1.0,Individual,148.0,128.0,1.0,0.0,1
1,68355089,24700.0,36 months,11.99,820.28,5,C1,engineer,6,65000.0,Not Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,small_business,business,577xx,SD,16.06,Dec-1999,22.0,0.0,21470.0,19.2,38.0,w,1.0,Individual,113.0,192.0,4.0,0.0,1
2,68341763,20000.0,60 months,10.78,432.66,6,B4,truck driver,6,63000.0,Not Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,unknown,605xx,IL,10.78,Aug-2000,6.0,0.0,7869.0,56.2,18.0,w,1.0,Joint App,125.0,184.0,5.0,0.0,1
3,68476807,10400.0,60 months,22.45,289.91,2,F1,contract specialist,6,104433.0,Source Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,major_purchase,major purchase,174xx,PA,25.37,Jun-1998,12.0,0.0,21929.0,64.5,35.0,w,1.0,Individual,128.0,210.0,6.0,0.0,1
4,68426831,11950.0,36 months,13.44,405.18,5,C3,veterinary tecnician,5,34000.0,Source Verified,2015-12-01,Fully Paid,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,debt consolidation,300xx,GA,10.20,Oct-1987,5.0,0.0,8822.0,68.4,6.0,w,1.0,Individual,338.0,54.0,0.0,0.0,1


In [38]:
features = ['loan_amnt','int_rate','installment','annual_inc','dti','grade','home_ownership',\
            'total_acc','open_acc','pub_rec','revol_bal','revol_util','mort_acc','paid_flag',\
            'mo_sin_old_il_acct','mo_sin_old_rev_tl_op'] #'sub_grade' #selecting final features #'addr_state''tax_liens',
loans = loans[features] #16 features with target var
loans = loans.reset_index(drop=True)
print("Current shape of dataset :",loans.shape)

Current shape of dataset : (475000, 16)


In [39]:
loans.head()

,loan_amnt,int_rate,installment,annual_inc,dti,grade,home_ownership,total_acc,open_acc,pub_rec,revol_bal,revol_util,mort_acc,paid_flag,mo_sin_old_il_acct,mo_sin_old_rev_tl_op
0,3600.0,13.99,123.03,55000.0,5.91,5,6,13.0,7.0,0.0,2765.0,29.7,1.0,1,148.0,128.0
1,24700.0,11.99,820.28,65000.0,16.06,5,6,38.0,22.0,0.0,21470.0,19.2,4.0,1,113.0,192.0
2,20000.0,10.78,432.66,63000.0,10.78,6,6,18.0,6.0,0.0,7869.0,56.2,5.0,1,125.0,184.0
3,10400.0,22.45,289.91,104433.0,25.37,2,6,35.0,12.0,0.0,21929.0,64.5,6.0,1,128.0,210.0
4,11950.0,13.44,405.18,34000.0,10.20,5,5,6.0,5.0,0.0,8822.0,68.4,0.0,1,338.0,54.0


In [40]:
print(loans['paid_flag'].value_counts())

1    376186
0     98814
Name: paid_flag, dtype: int64


In [41]:
# dummify
target = loans['paid_flag']
loans = loans.drop(['paid_flag'], axis=1)

loans = pd.get_dummies(loans, drop_first=True)
loans_norm = preprocessing.normalize(loans)

# seprate training & test set
X_train, X_test, Y_train, Y_test = train_test_split(loans_norm, target, shuffle=True, test_size=0.2)

## Modeling

1. Logistic Regression with Grid Search


Logistic Regression when implemented with grid search we got the best accuracies and minimum false negatives. We used the randomized search to find the best hyper paramters for the model.

In [ ]:
from sklearn.model_selection import GridSearchCV
def cross_validation_best_parameters(model, param_grid):
    grid = GridSearchCV(model, param_grid,cv=10, scoring='accuracy')
    X=loans.iloc[:,:-1].values
    y=loans.iloc[:,-1].values
    grid.fit(X,y)
    mean_scores = [result.mean_validation_score for result in grid.grid_scores_]
    return mean_scores,grid.best_score_,grid.best_estimator_
logreg = linear_model.LogisticRegression(random_state=0)
c=[0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(C=c)
mean_scores,Best_Accuracy, Best_classifier = cross_validation_best_parameters(logreg,param_grid)
print("Best accuracy is "+ str(Best_Accuracy))
print(Best_classifier)

In [ ]:
clf_LR = linear_model.LogisticRegression(C=Best_classifier.C)
clf_LR.fit(X_train,y_train)
LR_Predict = clf_LR.predict_proba(X_test)[:,1]
LR_Predict_bin = clf_LR.predict(X_test)
LR_Accuracy = accuracy_score(y_test,LR_Predict.round())
print("Logistic regression accuracy is ",LR_Accuracy)
plotAUC(y_test,rfPredictproba, 'Random Forest')
plotAUC(y_test,LR_Predict,'Logistic Regression')
plt.show()
plt.figure(figsize=(6,6))
plot_confusion_matrix(LR_Predict_bin, normalize=True)
plt.show()

2. Random Forest with randomized Search


Random forest when implemented with randomized search we got the best accuracies and minimum false negatives(predicting borowwer will not default eventhough he will. This might impact on the credibility of the company). We used the randomized search to find the best hyper paramters for the model.

In [ ]:
from sklearn.grid_search import RandomizedSearchCV
rf = RandomForestClassifier(criterion='gini', random_state=0)
maxFeatures = range(1,data_clean.shape[1]-1)
param_dist = dict(max_features=maxFeatures)
rand = RandomizedSearchCV(rf, param_dist, cv=10, scoring='accuracy', n_iter=len(maxFeatures), random_state=10)
X=data_clean.iloc[:,:-1].values
y=data_clean.iloc[:,-1].values
rand.fit(X,y)
mean_scores = [result.mean_validation_score for result in rand.grid_scores_]
#print('Best Accuracy = '+str(rand.best_score_))
print(rand.best_estimator_)

In [ ]:
randomForest = RandomForestClassifier(bootstrap=True,criterion = "gini",max_features=rand.best_estimator_.max_features,random_state=0 )
randomForest.fit(X_train,y_train)
rfPredict = randomForest.predict(X_test)
rfPredictproba = randomForest.predict_proba(X_test)[:,1] #for ROC curve
rfAccuracy = accuracy_score(y_test,rfPredict)
roc_score = metrics.roc_auc_score(y_test,rfPredict)
print(rfAccuracy)

## Feature Importance

In [ ]:
fig, ax = plt.subplots()
width=0.35
ax.bar(np.arange(len(features)-1), randomForest.feature_importances_, width, color='r')
ax.set_xticks(np.arange(len(randomForest.feature_importances_)))
ax.set_xticklabels(X_train.columns.values,rotation=90)
plt.title('Feature Importance from DT')
ax.set_ylabel('Normalized Gini Importance')

In [ ]:
plotAUC(y_test,rfPredictproba, 'Random Forest')
plt.show()
plt.figure(figsize=(6,6))
plot_confusion_matrix(rfPredict, normalize=True)
plt.show()

## Support Vector Machines(SVM) with Grid Search CV